In [194]:
from elasticsearch import Elasticsearch

In [195]:
es= Elasticsearch("https://localhost:9200",
                  basic_auth=('elastic', 'UXjHr=I*dSOPFsfM3rIJ'),
                  verify_certs=False)
es.ping()
                  

/Users/shivani/Desktop/Machine learning/semantic/venv2/lib/python3.12/site-packages/elasticsearch/_sync/client/__init__.py:399: SecurityWarning: Connecting to 'https://localhost:9200' using TLS with verify_certs=False is insecure
  _transport = transport_class(
/Users/shivani/Desktop/Machine learning/semantic/venv2/lib/python3.12/site-packages/urllib3/connectionpool.py:1103: InsecureRequestWarning: Unverified HTTPS request is being made to host 'localhost'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


True

In [196]:
import pandas as pd
import numpy as np

In [197]:
movies = pd.read_csv("imdb_100.csv")


columns_to_drop = ['Poster_Link', 'Meta_score', 'No_of_Votes','Gross']
movies.head()

,Poster_Link,Series_Title,Released_Year,Certificate,Runtime,Genre,IMDB_Rating,Overview,Meta_score,Director,Star1,Star2,Star3,Star4,No_of_Votes,Gross
0,https://m.media-amazon.com/images/M/MV5BMDFkYT...,The Shawshank Redemption,1994,A,142 min,Drama,9.3,Two imprisoned men bond over a number of years...,80.0,Frank Darabont,Tim Robbins,Morgan Freeman,Bob Gunton,William Sadler,2343110,"28,341,469"
1,https://m.media-amazon.com/images/M/MV5BM2MyNj...,The Godfather,1972,A,175 min,"Crime, Drama",9.2,An organized crime dynasty's aging patriarch t...,100.0,Francis Ford Coppola,Marlon Brando,Al Pacino,James Caan,Diane Keaton,1620367,"134,966,411"
2,https://m.media-amazon.com/images/M/MV5BMTMxNT...,The Dark Knight,2008,UA,152 min,"Action, Crime, Drama",9.0,When the menace known as the Joker wreaks havo...,84.0,Christopher Nolan,Christian Bale,Heath Ledger,Aaron Eckhart,Michael Caine,2303232,"534,858,444"
3,https://m.media-amazon.com/images/M/MV5BMWMwMG...,The Godfather: Part II,1974,A,202 min,"Crime, Drama",9.0,The early life and career of Vito Corleone in ...,90.0,Francis Ford Coppola,Al Pacino,Robert De Niro,Robert Duvall,Diane Keaton,1129952,"57,300,000"
4,https://m.media-amazon.com/images/M/MV5BMWU4N2...,12 Angry Men,1957,U,96 min,"Crime, Drama",9.0,A jury holdout attempts to prevent a miscarria...,96.0,Sidney Lumet,Henry Fonda,Lee J. Cobb,Martin Balsam,John Fiedler,689845,"4,360,000"


In [198]:
movies.drop(columns=columns_to_drop, inplace=True)

movies.head()

movies['Cast'] = movies[['Star1', 'Star2', 'Star3', 'Star4']].apply(lambda x: ' , '.join(x[x.notnull()]), axis=1)

movies['Cast'] = movies['Star1'] + movies['Star2']+movies['Star3']+movies['Star4']

movies.drop(columns=['Star1','Star2','Star3','Star4'],inplace=True)

In [199]:
movies.head()

,Series_Title,Released_Year,Certificate,Runtime,Genre,IMDB_Rating,Overview,Director,Cast
0,The Shawshank Redemption,1994,A,142 min,Drama,9.3,Two imprisoned men bond over a number of years...,Frank Darabont,Tim RobbinsMorgan FreemanBob GuntonWilliam Sadler
1,The Godfather,1972,A,175 min,"Crime, Drama",9.2,An organized crime dynasty's aging patriarch t...,Francis Ford Coppola,Marlon BrandoAl PacinoJames CaanDiane Keaton
2,The Dark Knight,2008,UA,152 min,"Action, Crime, Drama",9.0,When the menace known as the Joker wreaks havo...,Christopher Nolan,Christian BaleHeath LedgerAaron EckhartMichael...
3,The Godfather: Part II,1974,A,202 min,"Crime, Drama",9.0,The early life and career of Vito Corleone in ...,Francis Ford Coppola,Al PacinoRobert De NiroRobert DuvallDiane Keaton
4,12 Angry Men,1957,U,96 min,"Crime, Drama",9.0,A jury holdout attempts to prevent a miscarria...,Sidney Lumet,Henry FondaLee J. CobbMartin BalsamJohn Fiedler


In [200]:
movies.info

<bound method DataFrame.info of                  Series_Title Released_Year Certificate  Runtime  \
0    The Shawshank Redemption          1994           A  142 min   
1               The Godfather          1972           A  175 min   
2             The Dark Knight          2008          UA  152 min   
3      The Godfather: Part II          1974           A  202 min   
4                12 Angry Men          1957           U   96 min   
..                        ...           ...         ...      ...   
995    Breakfast at Tiffany's          1961           A  115 min   
996                     Giant          1956           G  201 min   
997     From Here to Eternity          1953      Passed  118 min   
998                  Lifeboat          1944         NaN   97 min   
999              The 39 Steps          1935         NaN   86 min   

                        Genre  IMDB_Rating  \
0                       Drama          9.3   
1                Crime, Drama          9.2   
2        Acti

In [201]:
movies.isna().sum()

Series_Title       0
Released_Year      0
Certificate      101
Runtime            0
Genre              0
IMDB_Rating        0
Overview           0
Director           0
Cast               0
dtype: int64

In [202]:
movies.drop(columns=['Certificate'],inplace=True)

In [203]:
movies.drop(columns=['Runtime'],inplace=True)

In [204]:

movies.loc[movies.duplicated()]

,Series_Title,Released_Year,Genre,IMDB_Rating,Overview,Director,Cast


In [205]:
movies['Column_for_embedding'] = movies['Overview'] 
movies['Column_for_embedding'] = movies['Column_for_embedding'].str.lower()  # Convert to lowercase

movies.head()

,Series_Title,Released_Year,Genre,IMDB_Rating,Overview,Director,Cast,Column_for_embedding
0,The Shawshank Redemption,1994,Drama,9.3,Two imprisoned men bond over a number of years...,Frank Darabont,Tim RobbinsMorgan FreemanBob GuntonWilliam Sadler,two imprisoned men bond over a number of years...
1,The Godfather,1972,"Crime, Drama",9.2,An organized crime dynasty's aging patriarch t...,Francis Ford Coppola,Marlon BrandoAl PacinoJames CaanDiane Keaton,an organized crime dynasty's aging patriarch t...
2,The Dark Knight,2008,"Action, Crime, Drama",9.0,When the menace known as the Joker wreaks havo...,Christopher Nolan,Christian BaleHeath LedgerAaron EckhartMichael...,when the menace known as the joker wreaks havo...
3,The Godfather: Part II,1974,"Crime, Drama",9.0,The early life and career of Vito Corleone in ...,Francis Ford Coppola,Al PacinoRobert De NiroRobert DuvallDiane Keaton,the early life and career of vito corleone in ...
4,12 Angry Men,1957,"Crime, Drama",9.0,A jury holdout attempts to prevent a miscarria...,Sidney Lumet,Henry FondaLee J. CobbMartin BalsamJohn Fiedler,a jury holdout attempts to prevent a miscarria...


In [207]:
movies.isna().sum()

Series_Title            0
Released_Year           0
Genre                   0
IMDB_Rating             0
Overview                0
Director                0
Cast                    0
Column_for_embedding    0
dtype: int64

In [208]:
duplicate_rows = movies[movies.duplicated(subset=['Series_Title'])]
print(duplicate_rows)

    Series_Title Released_Year                  Genre  IMDB_Rating  \
136     Drishyam          2015  Crime, Drama, Mystery          8.2   

                                              Overview         Director  \
136  Desperate measures are taken by a man who trie...  Nishikant Kamat   

                                       Cast  \
136  Ajay DevgnShriya SaranTabuRajat Kapoor   

                                  Column_for_embedding  
136  desperate measures are taken by a man who trie...  


In [209]:
movies=movies.drop_duplicates(subset=['Series_Title'])

In [210]:
movies = movies.reset_index(drop=True)

In [211]:
movies.shape

(999, 8)

In [212]:
movies.isna().value_counts()

Series_Title  Released_Year  Genre  IMDB_Rating  Overview  Director  Cast   Column_for_embedding
False         False          False  False        False     False     False  False                   999
Name: count, dtype: int64

In [213]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('all-mpnet-base-v2')

In [214]:
movies['Embedded_vectors']= movies['Column_for_embedding'].apply(lambda x: model.encode(x))

In [215]:
movies.head()

,Series_Title,Released_Year,Genre,IMDB_Rating,Overview,Director,Cast,Column_for_embedding,Embedded_vectors
0,The Shawshank Redemption,1994,Drama,9.3,Two imprisoned men bond over a number of years...,Frank Darabont,Tim RobbinsMorgan FreemanBob GuntonWilliam Sadler,two imprisoned men bond over a number of years...,"[0.038463116, 0.08977322, 0.008078586, -0.0135..."
1,The Godfather,1972,"Crime, Drama",9.2,An organized crime dynasty's aging patriarch t...,Francis Ford Coppola,Marlon BrandoAl PacinoJames CaanDiane Keaton,an organized crime dynasty's aging patriarch t...,"[-0.004622497, 0.09761083, 0.019865781, 0.0733..."
2,The Dark Knight,2008,"Action, Crime, Drama",9.0,When the menace known as the Joker wreaks havo...,Christopher Nolan,Christian BaleHeath LedgerAaron EckhartMichael...,when the menace known as the joker wreaks havo...,"[-0.005756101, 0.037831146, -0.025874881, -0.0..."
3,The Godfather: Part II,1974,"Crime, Drama",9.0,The early life and career of Vito Corleone in ...,Francis Ford Coppola,Al PacinoRobert De NiroRobert DuvallDiane Keaton,the early life and career of vito corleone in ...,"[-0.053500697, 0.106327996, 0.0149310725, 0.04..."
4,12 Angry Men,1957,"Crime, Drama",9.0,A jury holdout attempts to prevent a miscarria...,Sidney Lumet,Henry FondaLee J. CobbMartin BalsamJohn Fiedler,a jury holdout attempts to prevent a miscarria...,"[0.046683654, 0.0087696165, 0.016110895, -0.00..."


In [216]:
es.ping()

/Users/shivani/Desktop/Machine learning/semantic/venv2/lib/python3.12/site-packages/urllib3/connectionpool.py:1103: InsecureRequestWarning: Unverified HTTPS request is being made to host 'localhost'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


True

In [224]:
from indexMapping import indexMapping
es.indices.create(index="all_ms",mappings=indexMapping)

/Users/shivani/Desktop/Machine learning/semantic/venv2/lib/python3.12/site-packages/urllib3/connectionpool.py:1103: InsecureRequestWarning: Unverified HTTPS request is being made to host 'localhost'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


BadRequestError: BadRequestError(400, 'resource_already_exists_exception', 'index [all_ms/yNK5vpukT_G7yB_lnkD3Zw] already exists')

In [225]:
records_list=movies.to_dict("records")

In [227]:
records_list[0]

{'Series_Title': 'The Shawshank Redemption',
 'Released_Year': '1994',
 'Genre': 'Drama',
 'IMDB_Rating': 9.3,
 'Overview': 'Two imprisoned men bond over a number of years, finding solace and eventual redemption through acts of common decency.',
 'Director': 'Frank Darabont',
 'Cast': 'Tim RobbinsMorgan FreemanBob GuntonWilliam Sadler',
 'Column_for_embedding': 'two imprisoned men bond over a number of years, finding solace and eventual redemption through acts of common decency.',
 'Embedded_vectors': array([ 3.84631157e-02,  8.97732228e-02,  8.07858631e-03, -1.35398870e-02,
        -5.76251782e-02,  3.51737738e-02,  2.04893220e-02, -3.00286077e-02,
         1.04082413e-02,  5.30122686e-03,  1.70910005e-02, -2.03135647e-02,
         7.21534491e-02, -5.32962978e-02, -3.33577804e-02, -5.86996600e-02,
        -1.77770085e-03,  5.54492027e-02, -6.45608315e-03,  8.09579447e-04,
         5.22737317e-02,  2.62620300e-02,  2.68044155e-02, -7.77280191e-03,
         2.25521997e-03, -2.32486706e-

In [221]:
es.ping()

/Users/shivani/Desktop/Machine learning/semantic/venv2/lib/python3.12/site-packages/urllib3/connectionpool.py:1103: InsecureRequestWarning: Unverified HTTPS request is being made to host 'localhost'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


True

In [192]:
es.ping()

/Users/shivani/Desktop/Machine learning/semantic/venv2/lib/python3.12/site-packages/urllib3/connectionpool.py:1103: InsecureRequestWarning: Unverified HTTPS request is being made to host 'localhost'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


True

In [228]:
for record in records_list:
    try:
        es.index(index="all_ms", document=record)
    except Exception as e:
        print(e)

/Users/shivani/Desktop/Machine learning/semantic/venv2/lib/python3.12/site-packages/urllib3/connectionpool.py:1103: InsecureRequestWarning: Unverified HTTPS request is being made to host 'localhost'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/Users/shivani/Desktop/Machine learning/semantic/venv2/lib/python3.12/site-packages/urllib3/connectionpool.py:1103: InsecureRequestWarning: Unverified HTTPS request is being made to host 'localhost'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/Users/shivani/Desktop/Machine learning/semantic/venv2/lib/python3.12/site-packages/urllib3/connectionpool.py:1103: InsecureRequestWarning: Unverified HTTPS request is being made to host 'localhost'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/adv

BadRequestError(400, 'document_parsing_exception', 'Failed to parse object: expecting token of type [VALUE_NUMBER] but found [FIELD_NAME]')
BadRequestError(400, 'document_parsing_exception', 'Failed to parse object: expecting token of type [VALUE_NUMBER] but found [FIELD_NAME]')
BadRequestError(400, 'document_parsing_exception', 'Failed to parse object: expecting token of type [VALUE_NUMBER] but found [FIELD_NAME]')
BadRequestError(400, 'document_parsing_exception', 'Failed to parse object: expecting token of type [VALUE_NUMBER] but found [FIELD_NAME]')
BadRequestError(400, 'document_parsing_exception', 'Failed to parse object: expecting token of type [VALUE_NUMBER] but found [FIELD_NAME]')
BadRequestError(400, 'document_parsing_exception', 'Failed to parse object: expecting token of type [VALUE_NUMBER] but found [FIELD_NAME]')
BadRequestError(400, 'document_parsing_exception', 'Failed to parse object: expecting token of type [VALUE_NUMBER] but found [FIELD_NAME]')
BadRequestError(400,

/Users/shivani/Desktop/Machine learning/semantic/venv2/lib/python3.12/site-packages/urllib3/connectionpool.py:1103: InsecureRequestWarning: Unverified HTTPS request is being made to host 'localhost'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/Users/shivani/Desktop/Machine learning/semantic/venv2/lib/python3.12/site-packages/urllib3/connectionpool.py:1103: InsecureRequestWarning: Unverified HTTPS request is being made to host 'localhost'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/Users/shivani/Desktop/Machine learning/semantic/venv2/lib/python3.12/site-packages/urllib3/connectionpool.py:1103: InsecureRequestWarning: Unverified HTTPS request is being made to host 'localhost'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/adv

BadRequestError(400, 'document_parsing_exception', 'Failed to parse object: expecting token of type [VALUE_NUMBER] but found [FIELD_NAME]')
BadRequestError(400, 'document_parsing_exception', 'Failed to parse object: expecting token of type [VALUE_NUMBER] but found [FIELD_NAME]')
BadRequestError(400, 'document_parsing_exception', 'Failed to parse object: expecting token of type [VALUE_NUMBER] but found [FIELD_NAME]')
BadRequestError(400, 'document_parsing_exception', 'Failed to parse object: expecting token of type [VALUE_NUMBER] but found [FIELD_NAME]')
BadRequestError(400, 'document_parsing_exception', 'Failed to parse object: expecting token of type [VALUE_NUMBER] but found [FIELD_NAME]')
BadRequestError(400, 'document_parsing_exception', 'Failed to parse object: expecting token of type [VALUE_NUMBER] but found [FIELD_NAME]')
BadRequestError(400, 'document_parsing_exception', 'Failed to parse object: expecting token of type [VALUE_NUMBER] but found [FIELD_NAME]')
BadRequestError(400,

/Users/shivani/Desktop/Machine learning/semantic/venv2/lib/python3.12/site-packages/urllib3/connectionpool.py:1103: InsecureRequestWarning: Unverified HTTPS request is being made to host 'localhost'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/Users/shivani/Desktop/Machine learning/semantic/venv2/lib/python3.12/site-packages/urllib3/connectionpool.py:1103: InsecureRequestWarning: Unverified HTTPS request is being made to host 'localhost'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/Users/shivani/Desktop/Machine learning/semantic/venv2/lib/python3.12/site-packages/urllib3/connectionpool.py:1103: InsecureRequestWarning: Unverified HTTPS request is being made to host 'localhost'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/adv

...

/Users/shivani/Desktop/Machine learning/semantic/venv2/lib/python3.12/site-packages/urllib3/connectionpool.py:1103: InsecureRequestWarning: Unverified HTTPS request is being made to host 'localhost'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/Users/shivani/Desktop/Machine learning/semantic/venv2/lib/python3.12/site-packages/urllib3/connectionpool.py:1103: InsecureRequestWarning: Unverified HTTPS request is being made to host 'localhost'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/Users/shivani/Desktop/Machine learning/semantic/venv2/lib/python3.12/site-packages/urllib3/connectionpool.py:1103: InsecureRequestWarning: Unverified HTTPS request is being made to host 'localhost'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/adv

BadRequestError(400, 'document_parsing_exception', 'Failed to parse object: expecting token of type [VALUE_NUMBER] but found [FIELD_NAME]')
BadRequestError(400, 'document_parsing_exception', 'Failed to parse object: expecting token of type [VALUE_NUMBER] but found [FIELD_NAME]')
BadRequestError(400, 'document_parsing_exception', 'Failed to parse object: expecting token of type [VALUE_NUMBER] but found [FIELD_NAME]')
BadRequestError(400, 'document_parsing_exception', 'Failed to parse object: expecting token of type [VALUE_NUMBER] but found [FIELD_NAME]')
BadRequestError(400, 'document_parsing_exception', 'Failed to parse object: expecting token of type [VALUE_NUMBER] but found [FIELD_NAME]')
BadRequestError(400, 'document_parsing_exception', 'Failed to parse object: expecting token of type [VALUE_NUMBER] but found [FIELD_NAME]')
BadRequestError(400, 'document_parsing_exception', 'Failed to parse object: expecting token of type [VALUE_NUMBER] but found [FIELD_NAME]')
BadRequestError(400,

/Users/shivani/Desktop/Machine learning/semantic/venv2/lib/python3.12/site-packages/urllib3/connectionpool.py:1103: InsecureRequestWarning: Unverified HTTPS request is being made to host 'localhost'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/Users/shivani/Desktop/Machine learning/semantic/venv2/lib/python3.12/site-packages/urllib3/connectionpool.py:1103: InsecureRequestWarning: Unverified HTTPS request is being made to host 'localhost'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/Users/shivani/Desktop/Machine learning/semantic/venv2/lib/python3.12/site-packages/urllib3/connectionpool.py:1103: InsecureRequestWarning: Unverified HTTPS request is being made to host 'localhost'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/adv

BadRequestError(400, 'document_parsing_exception', 'Failed to parse object: expecting token of type [VALUE_NUMBER] but found [FIELD_NAME]')
BadRequestError(400, 'document_parsing_exception', 'Failed to parse object: expecting token of type [VALUE_NUMBER] but found [FIELD_NAME]')
BadRequestError(400, 'document_parsing_exception', 'Failed to parse object: expecting token of type [VALUE_NUMBER] but found [FIELD_NAME]')
BadRequestError(400, 'document_parsing_exception', 'Failed to parse object: expecting token of type [VALUE_NUMBER] but found [FIELD_NAME]')
BadRequestError(400, 'document_parsing_exception', 'Failed to parse object: expecting token of type [VALUE_NUMBER] but found [FIELD_NAME]')
BadRequestError(400, 'document_parsing_exception', 'Failed to parse object: expecting token of type [VALUE_NUMBER] but found [FIELD_NAME]')
BadRequestError(400, 'document_parsing_exception', 'Failed to parse object: expecting token of type [VALUE_NUMBER] but found [FIELD_NAME]')
BadRequestError(400,

/Users/shivani/Desktop/Machine learning/semantic/venv2/lib/python3.12/site-packages/urllib3/connectionpool.py:1103: InsecureRequestWarning: Unverified HTTPS request is being made to host 'localhost'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/Users/shivani/Desktop/Machine learning/semantic/venv2/lib/python3.12/site-packages/urllib3/connectionpool.py:1103: InsecureRequestWarning: Unverified HTTPS request is being made to host 'localhost'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/Users/shivani/Desktop/Machine learning/semantic/venv2/lib/python3.12/site-packages/urllib3/connectionpool.py:1103: InsecureRequestWarning: Unverified HTTPS request is being made to host 'localhost'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/adv

In [223]:
es.count(index="all_movie")

/Users/shivani/Desktop/Machine learning/semantic/venv2/lib/python3.12/site-packages/urllib3/connectionpool.py:1103: InsecureRequestWarning: Unverified HTTPS request is being made to host 'localhost'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


ObjectApiResponse({'count': 0, '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0}})

In [ ]:
input_keyword = "Comedy"
vector_input=model.encode(input_keyword)

query = {
    "field":"Embedded_vectors",
    "query_vector":
}